In [1]:
import datetime
import pandas as pd
import numpy as np
import psycopg2

#sensordata = pd.read_csv(sensorcsv)
#userdata = pd.read_csv(usercsv)

In [24]:
try:
    connection = psycopg2.connect(database ='heatseek', user = 'heatseek', password = 'wearecoolsoweseekheat')
    cursor = connection.cursor() #Open a cursor to perform operations
    cursor.execute('SELECT * from users;') #Executes the query
    users = cursor.fetchall() #cursor.fetchone() for one line, fetchmany() for multiple lines, fetchall() for all lines
    users = pd.DataFrame(users) #Saves 'users' as a pandas dataframe

    cursor.close() 
    connection.close()
except psycopg2.DatabaseError, error:
    print 'Error %s' % error

      0           1           2   \
0     20  2014-05-08  2015-03-05   
1     15  2014-04-04  2015-04-06   
2    157  2015-08-27  2015-08-27   
3      9  2014-04-03  2015-03-05   
4     25  2014-09-05  2015-04-30   
5    159  2015-09-04  2015-09-09   
6    164  2015-10-26  2015-11-02   
7     17  2014-04-07  2015-03-05   
8    152  2015-03-29  2015-04-19   
9    161  2015-10-19  2015-12-09   
10     5  2014-03-21  2014-04-03   
11   110  2014-12-17  2016-01-15   
12     8  2014-04-03  2015-04-08   
13   112  2014-12-18  2016-01-26   
14     3  2014-03-19  2014-04-04   
15   162  2015-10-19  2015-10-26   
16   155  2015-07-26  2015-07-26   
17   156  2015-08-26  2015-09-08   
18   154  2015-07-22  2015-09-09   
19    21  2014-05-12  2014-05-27   
20   158  2015-09-04  2015-09-10   
21   160  2015-10-06  2015-11-02   
22    19  2014-05-07  2014-09-05   
23     1  2014-03-14  2014-09-02   
24   165  2015-10-26  2015-10-27   
25   163  2015-10-22  2015-11-02   
26   166  2015-11-04  2015-1

In [319]:
#sensordata.user_id.unique()
#sensordata.user_id.value_counts()
#sensordata.violation.value_counts() #This returns the number of 't's and 'f's
#np.sort(userdata.id.unique())
#np.intersect1d(userdata.id.unique(), sensordata.user_id.unique()) #Returns the common ids in both the datasets.

array([ 98,  25,   3,  87, 113, 114, 130, 115, 119,  26,   1,   5, 172,
       126, 127, 132, 134, 136, 137, 138, 135,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
       150, 124, 125, 123, 147, 145, 144, 143, 146,   7,  11,  12,  13,
        14, 154, 155, 159, 158,  23, 160, 161, 163, 162, 165,  89,  92,
        91,  90, 164,  88, 167, 166, 168, 169, 170, 112], dtype=int64)

In [322]:
#Triage
#We want to have a measure of which users are facing the most chronic problems.

#Write some code that subsets all the violation == 't' cases
sensordataviolations = sensordata[sensordata.violation == 't'] #here it is.

#Hackiest method: just number of violations/numberof nonviolations and sort users by that
#That is, which users have had the most violations given the total number of readings 

violationsovertime = []

for i in sensordata.user_id.unique():
    nonviolations = sensordata.loc[sensordata.user_id == i, 'violation'].value_counts()['f'] #Number of violations = 'f'
    try:
        violations = sensordata.loc[sensordata.user_id == i, 'violation'].value_counts()['t'] #Number of violations = 't'
    except KeyError:
        violations = 0    
    sensordata.loc[sensordata.user_id == i, 'vfreq'] = float(violations)/float(nonviolations)
    violationsovertime.append([i, (float(violations)/float(nonviolations))])

#violations over time gives first the user_id, then the proportion of how many of their readings are violations

In [321]:
#Data cleaning

#Getting rid of test cases:
#1. Can we just delete by test IDs?
#2. If testing was separated by a minute, we can find all test cases by looping through all users,
#  and if they have a bunch of data that was collected within minutes, delete the user?

#We first convert the string dates into datetime format
sensordata['formatteddate'] = sensordata.created_at.apply(lambda x: pd.to_datetime(x,  format = "%Y-%m-%d %H:%M"))

#Then, one way of telling if a user_id was an actual user or a test case was to calculate the average timedelta for each user_id.
#Timedeltas of 1min are tests, 1 hour are users (don't know if this is always true, but if no user has an average polling rate
#of 1 min, we can use a bunch of methods to filter away test cases step by step).

sensordata['averagetimedelta'] = 0.00 #makes a new column

for i in sensordata.user_id.unique(): #for each user
    timelist = sensordata.loc[sensordata.user_id == i, 'formatteddate'] #this gives us a list of all their times in timestamp
    timedeltas = [] 
    for j in range(1, len(timelist)-1):
        timedeltas.append(timelist.iloc[j] - timelist.iloc[j-1]) #list of differences in time between time point j and j-1
    try:
        #we print the user_id, followd by their average time delta from point j to j-1
        print i, abs(sum(timedeltas, datetime.timedelta(0)))/len(timedeltas) #the average timedelta
    except ZeroDivisionError: #some cases have too few points (and results in a zero division error)
        #instead of breaking with we encounter a zerodivisionerror, just print the following:
        print i, "Too few data points?"
    sensordata.loc[sensordata.user_id == i, 'averagetimedelta'] = averagetimedeltas.total_seconds()

#3. If user ids are recycled, we'll have to do a combination of those things.

#Some sensors

98 0 days 00:00:00.011915
25 0 days 00:48:36.900000
3 0 days 06:04:00.262110
87 0 days 13:55:44.925653
113 0 days 00:59:59.972928
114 0 days 01:01:14.343489
130 0 days 00:53:36.506154
115 0 days 01:00:59.473623
119 0 days 01:00:36.331119
26 0 days 00:00:00.000031
1 0 days 00:46:15.231805
5 0 days 00:49:50.485335
172 0 days 00:19:12.928003
126 0 days 00:06:28.098409
127 0 days 01:01:35.454545
132 0 days 00:43:52.864636
134 0 days 02:12:07.492227
136 0 days 00:03:49.787234
137 0 days 01:00:58.096774
138 0 days 00:27:27.259259
135 0 days 00:58:54.997742
34 0 days 01:00:00
35 0 days 01:00:00
36 0 days 01:00:00
37 0 days 01:00:00
38 0 days 01:00:00
39 0 days 01:00:00
40 0 days 01:00:00
41 0 days 01:00:00
42 0 days 01:00:00
43 0 days 01:00:00
44 0 days 01:00:00
45 0 days 01:00:00
46 0 days 01:00:00
47 0 days 01:00:00
48 0 days 01:00:00
49 0 days 01:00:00
50 0 days 01:00:00
51 0 days 01:00:00
52 0 days 01:00:00
53 0 days 01:00:00
54 0 days 01:00:00
55 0 days 01:00:00
56 0 days 01:00:00
57 0 d

In [ ]:
#Stuff to do for fun

#Variability/consistency
#Which buildings have the least/most variable temperatures?
#For this, we just calculate within-person variability (how much do sensor temperatures by the same user) vary as a function of time
#We an use this same process to calculate variability between locations (e.g., just calculate variance for each location)

In [ ]:
#Now, we loop over all unique users in the dataset and generate a measure of how long they've had the sensor running
sensordata['totaltime'] = 0
sensordata['vfreq'] = 0

for i in sensordata.user_id.unique():
    firstentry = len(sensordata.loc[sensordata.user_id==i,'formatteddate']) #This gives us the index of the first timepoint
    lasttime = sensordata.loc[sensordata.user_id == i, 'formatteddate'].iloc[0] #This was the timestamp of the latest timepoint
    firsttime =  sensordata.loc[sensordata.user_id == i, 'formatteddate'].iloc[firstentry-1] #This was the timestamp of the first timepoint
    sensordata.loc[sensordata.user_id == i, 'totaltime'] = lasttime - firsttime #This is the timedelta (over how long a period readings were made)
    #print i, lasttime-firsttime